# MetPy Calculations

In [ ]:
from datetime import datetime, timedelta

import metpy.calc as mpcalc
from metpy.plots.declarative import *
from metpy.units import units
import xarray as xr

In [ ]:
# Set the date/time of the model run
date = datetime(2020, 2, 28, 0)

# Remote access to the dataset from the UCAR site
ds = xr.open_dataset('https://thredds.ucar.edu/thredds/dodsC/grib'
                     f'/NCEP/GFS/Global_onedeg/GFS_Global_onedeg_{date:%Y%m%d}_{date:%H%M}.grib2')

# Subset data to be just over the U.S. for plotting purposes
ds = ds.sel(lat=slice(70,10), lon=slice(360-150, 360-55))

## A Simple Calculation

1000-500-hPa Thickness = (Heights at 500 hPa) - (Heights at 1000 hPa)

Easy to do subtraction with a data array and store it directly back into the dataset (ds).
                   

In [ ]:
ds['thickness'] = (ds.Geopotential_height_isobaric.metpy.sel(vertical=500 * units.hPa)
                   - ds.Geopotential_height_isobaric.metpy.sel(vertical=1000 * units.hPa))

In [ ]:
# Set the plot time with forecast hours
plot_time = date + timedelta(hours=0)

# Set attributes for plotting contours
cntr = ContourPlot()
cntr.data = ds
cntr.field = 'thickness'
cntr.level = None
cntr.time = plot_time
cntr.contours = list(range(0, 10000, 60))
cntr.linecolor = 'red'
cntr.linestyle = 'dashed'
cntr.clabels = True

cntr2 = ContourPlot()
cntr2.data = ds
cntr2.field = 'Geopotential_height_isobaric'
cntr2.level = 500 * units.hPa
cntr2.time = plot_time
cntr2.contours = list(range(0, 10000, 60))
cntr2.linecolor = 'black'
cntr2.linestyle = 'solid'
cntr2.clabels = True

# Set the attributes for the map
# and put the contours on the map
panel = MapPanel()
panel.area = [-125, -74, 20, 55]
panel.projection = 'lcc'
panel.layers = ['states', 'coastline', 'borders']
panel.title = f'1000-500-hPa Thickness at {plot_time} by KHG'
panel.plots = [cntr, cntr2]

# Set the attributes for the panel
# and put the panel in the figure
pc = PanelContainer()
pc.size = (15, 15)
pc.panels = [panel]

# Show the figure
pc.show()

## A Second Simple Calculation

Scaling Absolute Vorticity

Since vorticity values are on the order of 10^-5 we want to scale them to fit common contouring values of whole numbers.

In [ ]:
ds['Absolute_vorticity_isobaric'] = ds.Absolute_vorticity_isobaric * 1e5

In [ ]:
# Set the plot time with forecast hours
plot_time = date + timedelta(hours=0)

# Set attributes for plotting contours
cfill = FilledContourPlot()
cfill.data = ds
cfill.field = 'Absolute_vorticity_isobaric'
cfill.level = 500 * units.hPa
cfill.time = plot_time
cfill.contours = list(range(-40, 41, 2))
cfill.colormap = 'PuOr_r'
cfill.colorbar = 'horizontal'

cntr2 = ContourPlot()
cntr2.data = ds
cntr2.field = 'Geopotential_height_isobaric'
cntr2.level = 500 * units.hPa
cntr2.time = plot_time
cntr2.contours = list(range(0, 10000, 60))
cntr2.linecolor = 'black'
cntr2.linestyle = 'solid'
cntr2.clabels = True

# Set the attributes for the map
# and put the contours on the map
panel = MapPanel()
panel.area = [-125, -74, 20, 55]
panel.projection = 'lcc'
panel.layers = ['states', 'coastline', 'borders']
panel.title = f'{cfill.level.m}-hPa Abs. Vorticity at {plot_time} by KHG'
panel.plots = [cfill, cntr2]

# Set the attributes for the panel
# and put the panel in the figure
pc = PanelContainer()
pc.size = (15, 15)
pc.panels = [panel]

# Show the figure
pc.show()

## More Complex Calculations

1. Do the calculation using MetPy (not too hard).
2. Add it to the dataarray (a little more complicated).

All MetPy Calculations can be found at https://unidata.github.io/MetPy/v1.0/api/generated/metpy.calc.html#module-metpy.calc

Not all calculations work on grids, yet!

To calculate wind speed: https://unidata.github.io/MetPy/v1.0/api/generated/metpy.calc.wind_speed.html#metpy.calc.wind_speed

Assigning a new variable to our dataset requires a few pieces of information:
1. Named dimensions (easiest to use the dimension names from a similar variable already in the dataset)
2. The calculated values
3. Since our data is on a projection, we need to add that information to data array (can use any variable from dataset to set this value)
4. add the appropriate units, which can be obtained from the calculated variable from MetPy

```python
ds = ds.assign(<new_dataarray_name>=(<tuple of dim names>, <values_of_calculated_variable>,
                                     <dictionary of attributes including 'grid_mapping' and 'units'))

```

In [ ]:
# Calculate Wind Speed
wspd = mpcalc.wind_speed(ds['u-component_of_wind_isobaric'], ds['v-component_of_wind_isobaric'])

In [ ]:
# Add it to dataset (ds)

# First set needed elements based on what is being calculated
dim_names = ds['u-component_of_wind_isobaric'].dims
variable_values = wspd.m
attribute_dictionary = {'grid_mapping': ds['u-component_of_wind_isobaric'].grid_mapping,
                        'units': wspd.units}
# Add it to dataset
ds = ds.assign(wind_speed=(dim_names, variable_values, attribute_dictionary))

In [ ]:
# Set the plot time with forecast hours
plot_time = date + timedelta(hours=0)

# Set attributes for plotting contours
cfill = FilledContourPlot()
cfill.data = ds
cfill.field = 'wind_speed'
cfill.level = 300 * units.hPa
cfill.time = plot_time
cfill.contours = list(range(10, 101, 10))
cfill.colormap = 'BuPu'
cfill.colorbar = 'horizontal'

cntr2 = ContourPlot()
cntr2.data = ds
cntr2.field = 'Geopotential_height_isobaric'
cntr2.level = 300 * units.hPa
cntr2.time = plot_time
cntr2.contours = list(range(0, 10000, 120))
cntr2.linecolor = 'black'
cntr2.linestyle = 'solid'
cntr2.clabels = True

# Set the attributes for the map
# and put the contours on the map
panel = MapPanel()
panel.area = [-125, -74, 20, 55]
panel.projection = 'lcc'
panel.layers = ['states', 'coastline', 'borders']
panel.title = f'{cfill.level.m}-hPa Heights and Wind Speed at {plot_time} by KHG'
panel.plots = [cfill, cntr2]

# Set the attributes for the panel
# and put the panel in the figure
pc = PanelContainer()
pc.size = (15, 15)
pc.panels = [panel]

# Show the figure
pc.show()